In [ ]:
import math
import numpy as np
from tqdm import tqdm
import json
import io
import statistics
import json
from google.colab import files
import requests
import pandas as pd
import requests
import time
import re
from scipy.stats import chisquare
from scipy import stats


In [ ]:

#The user list was updated to cover about 50 users at a time, which was changed manually between runs
user_list = ['1018470', '40743732', '42299496', '27928756', '19532902', '18917113', '48352335', '37816302', '54114912', '32132091', '42244366', '6012990', '48716577', '25242092', '43964689', '43208694', '6598883', '6448894', '2794854', '54038553', '28450113', '51395861', '31032891', '52007576', '12750271', '38750669', '35542574', '56365106', '17068474', '56535048', '9585434', '46717618', '32219293', '56494159', '51771869', '16234346', '56145313', '9025553', '6912015', '41145759', '15654949', '25048617', '53499495', '49529213', '7983444', '52420570', '29923505', '39384088', '45762493', '43615333', '55703498', '51381912', '34146527', '357970', '3332590', '22556307', '27711129', '21436880', '12037851', '11636112', '434502', '3904444', '3387006', '19469020', '56664962', '51783078', '28882579', '38861962', '25489959', '3945717', '58408695', '50320075', '22530867', '5187826', '36992120', '29758251', '2382900', '28889169', '37042408', '50200521', '22687175', '18799884', '9633031', '53991256', '27250247', '50290108', '40258374', '51977746', '3954145', '15535181', '1286172', '34559681', '9756440', '10059075', '22082433', '15789997', '23017592', '7243691', '1302798', '6023826']
while user_id != "6023826":
    user_id = user_list[step]
    user_reviews = {}
    start_id = "0"
    review_count = int(json.loads(requests.get(
                        "https://www.vivino.com/api/users/" + user_id,
                        headers= {
                            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                            "x-requested-with": "XMLHttpRequest"
                        }).text)["user"]["statistics"]["reviews_count"])
    if review_count < 500:
        while True:
            r = requests.get(
            "https://www.vivino.com/users/" + user_id + "/activities",
            params = {
                "limit":"10",
                "start_from_id":start_id
            },
            headers= {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                "accept": "text/javascript, application/javascript, application/ecmascript, application/x-ecmascript, */*; q=0.01",
                "x-requested-with": "XMLHttpRequest"
            }
            )

            vintage_list = []
            for item in r.text.split("data-vintage-id="):
                vintage_list.append(item.split(" ")[0][2:-2])
            vintage_list.pop(0)
            if len(vintage_list) == 0:
                break

            for vintage_id in vintage_list:
                try:
                    rating_json = json.loads(requests.get(
                            "https://www.vivino.com/api/users/" + user_id + "/vintages/" + vintage_id + "/reviews",
                            headers= {
                                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
                            }).text)["reviews"][0]["rating"]
                except:
                    rating_json = "N/A"
                try:
                    winery_json = (json.loads(( requests.get(
                            "https://www.vivino.com/api/vintages/" + vintage_id,
                            headers= {
                                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
                            }).text))["vintage"]["wine"]["winery"]["name"])
                except:
                    winery_json = "N/A"
                try:
                    price_json = json.loads((requests.get(
                            "https://www.vivino.com/api/checkout_prices/",
                            params = {
                                "vintage_id": vintage_id
                            },
                            headers= {
                                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
                            }).text))["prices"]["availability"]["median"]["amount"]
                except:
                    price_json = "N/A"

                review_json = json.dumps({
                    "rating" : rating_json,
                    "winery" : winery_json,
                    "price" : price_json
                })
                user_reviews[vintage_id] = review_json
            if len(vintage_list) == 10:
                start_id = re.sub('[")]',"",((r.text.split("last-id', ")[1]).split(";")[0]))
            else:
                break
        with open('user_reviews_'+user_id+'.json', 'w') as f:
            json.dump(user_reviews, f)
        print(user_id + " json done")
  
    step+=1

In [ ]:
user_list = ['3818162', '823866', '535199', '1004623', '22174877', '45879712', '37201917', '21184836', '5115332', '21643331', '33080613', '4771509', '9102620', '44033453', '33699739', '3123879', '5582650', '26700176', '8019507', '1011187', '38359529', '7972237', '10240263', '51546378', '33963540', '43012696', '10505362', '1814649', '37209041', '5769599', '32447287', '28748143', '11251730', '37118192', '22795263', '14009715', '48687505', '40311124', '3252020', '1060471', '19782668', '375328', '25744166', '2312301', '2160869', '21310174', '2729967', '25379640', '40784011', '46765836', '4154093', '46967764', '27382841', '51765602', '12642664', '12604542', '41935834', '3214850', '4054718', '3046972', '24283729', '8138410', '4184517', '21760903', '48099965', '44032555', '3009591', '42170806', '40370300', '3440907', '10380099', '37949388', '22815529', '17699380', '8862066', '5183924', '568115', '10438170', '33209600', '16895921', '27511159', '4769276', '3864830', '36114657', '38510902', '31158349', '50720353', '41433868', '43207177', '11694330', '10360336', '608011', '52764411', '55474091', '48670081', '49088250', '51822718', '34964871', '51787898', '41339835', '35869300', '34556055', '14459771', '5820714', '51447737', '31174241', '49932218', '33555024', '13238629', '34737182', '54163779', '32172218', '52358023', '42500519', '48394006', '13179756', '31499936', '48406692', '745243', '34581579', '1610847', '53962537', '56805084', '53457253', '52215556', '50180759', '35874504', '48823709', '1505867', '13853907', '30030135', '56349864', '29907908', '56149421', '36530109', '31291907', '54123470', '58880729', '10963657', '41274240', '37441116', '58259085', '49937910', '59334004', '12881874', '56175416', '34770105', '54116383', '41141382', '20956114', '1018470', '40743732', '42299496', '27928756', '19532902', '18917113', '48352335', '37816302', '54114912', '32132091', '42244366', '6012990', '48716577', '25242092', '43964689', '43208694', '6598883', '6448894', '2794854', '54038553', '28450113', '51395861', '31032891', '52007576', '12750271', '38750669', '35542574', '56365106', '17068474', '56535048', '9585434', '46717618', '32219293', '56494159', '51771869', '16234346', '56145313', '9025553', '6912015', '41145759', '15654949', '25048617', '53499495', '49529213', '7983444', '52420570', '29923505', '39384088', '45762493', '43615333', '55703498', '51381912', '34146527', '357970', '3332590', '22556307', '27711129', '21436880', '12037851', '11636112', '434502', '3904444', '3387006', '19469020', '56664962', '51783078', '28882579', '38861962', '25489959', '3945717', '58408695', '50320075', '22530867', '5187826', '36992120', '29758251', '2382900', '28889169', '37042408', '50200521', '22687175', '18799884', '9633031', '53991256', '27250247', '50290108', '40258374', '51977746', '3954145', '15535181', '1286172', '34559681', '9756440', '10059075', '22082433', '15789997', '23017592', '7243691', '1302798', '6023826']
#user_list exists
user_dictionary = {}
test = 0
for id in user_list:
    try:
        user_review_data = json.load(open('user_reviews_'+id+'.json'))
        wine_id_keys = list(user_review_data.keys())
        test += 1
        print(id)
        found_review = True
    except:
        found_review = False
    if found_review and 10 < len(list(user_review_data.keys())):
        print(id,' ok')
        winery_counter = {}
        for i in range(len(wine_id_keys)):
            review_data = json.loads(user_review_data[wine_id_keys[i]])
            winery = review_data["winery"]
            if winery in list(winery_counter.keys()):
                winery_counter[winery] +=1
            else:
                winery_counter[winery] = 1
        if "N/A" in  list(winery_counter.keys()):
            del winery_counter["N/A"]
        sorted_winery_counter = sorted(winery_counter.items(), key=lambda x: x[1], reverse=True)
        fav_winery = sorted_winery_counter[0][0]
        fav_winery_rating_tot = []
        fav_winery_price_tot = []
        other_wineries_rating_tot = []
        other_wineries_price_tot = []

        for i in range(len(wine_id_keys)):
            review_data = json.loads(user_review_data[wine_id_keys[i]])
            winery = review_data["winery"]
            price = review_data["price"]
            rating = review_data["rating"]
            if winery == fav_winery:
                fav_winery_price_tot.append(price)
                fav_winery_rating_tot.append(rating)
            else:
                other_wineries_price_tot.append(price)
                other_wineries_rating_tot.append(rating)
        fav_winery_rating_tot = [value for value in fav_winery_rating_tot if value != "N/A"]
        fav_winery_price_tot = [value for value in fav_winery_price_tot if value != "N/A"]
        other_wineries_rating_tot = [value for value in other_wineries_rating_tot if value != "N/A"]
        other_wineries_price_tot = [value for value in other_wineries_price_tot if value != "N/A"]
        if len(fav_winery_rating_tot) > 2 and len(fav_winery_price_tot) > 2:
            user_dictionary[id] = [statistics.mean(fav_winery_rating_tot), math.sqrt(statistics.variance(fav_winery_rating_tot)), 
                                statistics.mean(fav_winery_price_tot), math.sqrt(statistics.variance(fav_winery_price_tot)), 
                                statistics.mean(other_wineries_rating_tot), math.sqrt(statistics.variance(other_wineries_rating_tot)),
                                statistics.mean(other_wineries_price_tot), math.sqrt(statistics.variance(other_wineries_price_tot))]

with open("user_reviews_dictonary.json", "w") as outfile:
    json.dump(user_dictionary, outfile)

In [ ]:
f = open('user_reviews_dictonary.json')

data = json.load(f)
values_matrix = np.matrix(list(data.values()))


fav_rating_mean = []
fav_rating_sd = []
fav_price_mean = []
fav_price_sd =[]
other_rating_mean = []
other_rating_sd = []
other_price_mean = []
other_price_sd = []

for i in range(len(values_matrix)):

    fav_rating_mean.append(values_matrix[i,0]) 
    fav_rating_sd.append(values_matrix[i,1])  
    fav_price_mean.append(values_matrix[i,2]) 
    fav_price_sd.append(values_matrix[i,3]) 
    other_rating_mean.append(values_matrix[i,4])  
    other_rating_sd.append(values_matrix[i,5])  
    other_price_mean.append(values_matrix[i,6])  
    other_price_sd.append(values_matrix[i,7])  

print("Fav rating mean: " + str(statistics.mean(fav_rating_mean)))
print("Other rating mean: " + str(statistics.mean(other_rating_mean)))
print(" ")
print("Fav rating sd: " + str(statistics.mean(fav_rating_sd)))
print("Other rating sd: " + str(statistics.mean(other_rating_sd)))
print(" ")
print("Fav price mean: " + str(statistics.mean(fav_price_mean)))
print("Other price mean: " + str(statistics.mean(other_price_mean)))
print(" ")
print("Fav price sd: " + str(statistics.mean(fav_price_sd)))
print("Other price sd: " + str(statistics.mean(other_price_sd)))
